In [1]:
# pip install pyspark

     -------------------------------------- 281.4/281.4 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 199.7/199.7 kB ? eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=c6a6efe5e7bcea8ef4d9c2256fda8bf7f52688a7d96651fbc3c984d7d905ab1b
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\ad\fa\e3\0ed85d0d18ebd9e1344ffd6c1d1d0031af21eee1988ba36050
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

In [8]:
# Crea SparkSession
spark = SparkSession.builder.master("local[*]").appName("MiApp").getOrCreate()

# .master("local[*]") --> para que se usen todos los nucleos (cores) de tu PC

In [9]:
spark

In [12]:
sc = spark.sparkContext

**¿Qué es un RDD?**

Resilient Distributed Dataset (RDD)


In [13]:
# crear rdd vacio

rdd_vacio = sc.emptyRDD

In [14]:
# crear rdd vacio con 3 particiones

rdd_vacio_3 = sc.parallelize([], 3)

In [16]:
rdd_vacio3.getNumPartitions()

3

In [17]:
# crear rdd no vacio con 3 particiones
rdd_no_vacio_3 = sc.parallelize([1,2,3,4,5], 3)

In [18]:
# ver los datos almacenados en el rdd
rdd_no_vacio_3.collect()

[1, 2, 3, 4, 5]

In [23]:
# Crear un RDD desde un archivo de texto
rdd_texto = sc.textFile('./ejemplo_texto.txt')

In [24]:
rdd_texto.collect()

['Este es un ejemplo de archivo de texto :)',
 'Esto es una frase de prueba 1.',
 'Esto es una frase de prueba 2.',
 'Esto es una frase de prueba 3.',
 'Esto es una frase de prueba 4.',
 'Esto es una frase de prueba 5.']

In [25]:
rdd_texto_completo = sc.wholeTextFiles('./ejemplo_texto.txt')

In [26]:
rdd_texto_completo.collect()

[('file:/c:/Users/Usuario/Documents/fabio/Fabio/Estadistica4all.github.io-1/Notebooks/Spark/ejemplo_texto.txt',
  'Este es un ejemplo de archivo de texto :)\r\nEsto es una frase de prueba 1.\r\nEsto es una frase de prueba 2.\r\nEsto es una frase de prueba 3.\r\nEsto es una frase de prueba 4.\r\nEsto es una frase de prueba 5.')]

In [27]:
# Crear un RDD a partir de un data-frame:

data = spark.read.format("csv").option("header", "true").load("House_Price_Regression.csv")

rdd = sc.parallelize([], 3)

rdd_data = data.rdd

In [29]:
rdd_data.collect()[0:5]

[Row(neighborhood_recode='46.0', latitude='25.113208', longitude='55.138932', price='2700000', no_of_bedrooms='1', no_of_bathrooms='2', quality_recode='2.0', maid_room_recode='0.0', unfurnished_recode='0.0', balcony_recode='1.0', barbecue_area_recode='1.0', central_ac_recode='1.0', childrens_play_area_recode='1.0', childrens_pool_recode='0.0', concierge_recode='1.0', covered_parking_recode='0.0', kitchen_appliances_recode='1.0', maid_service_recode='0.0', pets_allowed_recode='1.0', private_garden_recode='0.0', private_gym_recode='0.0', private_jacuzzi_recode='0.0', private_pool_recode='0.0', security_recode='0.0', shared_gym_recode='1.0', shared_pool_recode='0.0', shared_spa_recode='0.0', view_of_water_recode='1.0', size_in_m_2='100.242337'),
 Row(neighborhood_recode='46.0', latitude='25.106809', longitude='55.151201', price='2850000', no_of_bedrooms='2', no_of_bathrooms='2', quality_recode='2.0', maid_room_recode='0.0', unfurnished_recode='0.0', balcony_recode='1.0', barbecue_area_rec

In [30]:
data.show(5)

+-------------------+---------+---------+-------+--------------+---------------+--------------+----------------+------------------+--------------+--------------------+-----------------+--------------------------+---------------------+----------------+----------------------+-------------------------+-------------------+-------------------+---------------------+------------------+----------------------+-------------------+---------------+-----------------+------------------+-----------------+--------------------+-----------+
|neighborhood_recode| latitude|longitude|  price|no_of_bedrooms|no_of_bathrooms|quality_recode|maid_room_recode|unfurnished_recode|balcony_recode|barbecue_area_recode|central_ac_recode|childrens_play_area_recode|childrens_pool_recode|concierge_recode|covered_parking_recode|kitchen_appliances_recode|maid_service_recode|pets_allowed_recode|private_garden_recode|private_gym_recode|private_jacuzzi_recode|private_pool_recode|security_recode|shared_gym_recode|shared_pool_r